In [90]:
from utils import *
from external_packages import *
import torch

In [91]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr_torch.models import DeepFM
from deepctr_torch.inputs import  SparseFeat, DenseFeat,get_fixlen_feature_names

In [92]:
final_stackset=pd.read_csv(processed_data_folder+'/final_stackset_PowerTransformer.csv',index_col=0)
final_stackset_features=final_stackset.drop(['ID','to_predict'],axis=1)
final_stackset_target=final_stackset.loc[:,['to_predict']]
final_stackset_target.groupby('to_predict')['to_predict'].count()

to_predict
0    999756
1       244
Name: to_predict, dtype: int64

In [93]:
dense_features = final_stackset.columns[1:]

In [94]:
final_stackset[dense_features] = final_stackset[dense_features].fillna(0,)

In [95]:
target = ['to_predict']

In [96]:
dense_feature_columns = [DenseFeat(feat, 1) for feat in dense_features]

In [107]:
feature_names=get_fixlen_feature_names(dense_feature_columns)

In [98]:
'to_predict' in feature_names

True

In [99]:
'ID' in feature_names

False

In [100]:
train, test =train_test_split(final_stackset[feature_names],
                        test_size=0.2,random_state =random_state )

In [101]:
train_model_input = [train[name] for name in feature_names]

test_model_input = [test[name] for name in feature_names]

In [109]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = DeepFM(dense_features, dense_features, task='binary',
               l2_reg_embedding=1e-5, device=device)

model.compile("adagrad", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"],)
model.fit(train_model_input, train[target].values,
          batch_size=32, epochs=10, validation_split=0.2, verbose=2)

AttributeError: 'str' object has no attribute 'name'